# RAG Inference

This notebook follows `rag_create_index.ipynb` where we set up a RAG index. Here we actually use that index to interact with RAG-enabled cosmosage.

In [ ]:
# load model, tokenizer, retriever
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from auto_gptq import AutoGPTQForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM

model_dir = "models/cosmosage_v2/"
#model = AutoGPTQForCausalLM.from_quantized(model_dir) # not yet supported
model = AutoModelForCausalLM.from_pretrained(model_dir, torch_dtype=torch.bfloat16).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_dir)
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-large-en-v1.5')
index = FAISS.load_local(f"datasets/faiss_index.bin", embeddings)
retriever = index.as_retriever(search_type="similarity", search_kwargs={'k': 4})

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.4,
    repetition_penalty=1.01,
    return_full_text=True,
    max_new_tokens=1000,
    do_sample=True,
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
prompt_template = """Assume the role of an expert cosmologist. You answer the USER's question clearly with sufficient context. Here is some context that might help:

{context}

USER: {question}
ASSISTANT:"""
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
llm_chain = LLMChain(llm=llm, prompt=prompt)
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [ ]:
question = "What does the D_SZ parameter correspond to in the SPT cluster abundance cosmology analysis?"

In [ ]:
result = llm_chain.invoke({"context":"", "question": question})
print(result['text'])

In [ ]:
result_rag = rag_chain.invoke(question)
print(result_rag['text'])

In [ ]:
result_rag